# Which Features Are Important?

An attempt to explore a few different ways to reason about important features. For the conf paper, we calculated both inpurity-based feature importance during training, and also permutation feature importance for trained models.

In [33]:
FIMP_FILE = "../data/xtra-berlin/feature-importance.csv.gz"

import itertools as it
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

fimps = pd.read_csv(FIMP_FILE)
print("original columns:",fimps.columns.tolist())

fromto = {r'sarofe_sums_' : 'li_',r'sarofe_pbs_' : 'pb_'}
for f,t in fromto.items():
    fimps['feature'] = fimps.feature.str.replace(re.compile(f),t)
fimps['featureset'] = fimps.featureset.str.replace(re.compile(r'sumpb'),'lipb')

pfi = fimps.loc[fimps.source=='perm',['split','featureset','cycle','feature','importance']]
pfi

original columns: ['split', 'featureset', 'cycle', 'confA', 'confB', 'feature', 'source', 'importance']


,split,featureset,cycle,feature,importance
2760,class,combi,10,c_avg_deg_cons,121.3246
2761,class,combi,10,c_avg_dom_cons,241.0968
2762,class,combi,10,c_avg_domdeg_cons,87.3698
2763,class,combi,10,c_bounds_d,0.0000
2764,class,combi,10,c_bounds_r,0.0000
...,...,...,...,...,...
739195,instance,lipb,9,li_skew_min,16.0702
739196,instance,lipb,9,li_skew_mn,55.6060
739197,instance,lipb,9,li_wsum_iqr,40.3666
739198,instance,lipb,9,li_wsum_skew,-17.4002


In [35]:
_imp = pfi.groupby(['feature','split','featureset'])['importance'].agg(['mean','median']).reset_index()


for task, fset, avg in it.product(_imp.split.unique().tolist(),_imp.featureset.unique().tolist(),('mean','median')):
    imp_here = _imp.loc[(_imp.split==task) & (_imp.featureset==fset)]
    print(task,fset,f'by {avg}')
    print(imp_here.sort_values(avg,ascending=False).iloc[:20][['feature','mean','median']].round(2),'\n')

class combi by mean
                    feature     mean  median
576                pb_k_med  6441.16   -1.57
150          c_sum_dom_cons  4561.58   -0.01
60           c_ent_deg_cons  4440.39   -4.40
592                pb_n_iqr  4235.50   -0.56
288     li_amogs_maxw_mn_mn  3755.80    0.00
120       c_min_domdeg_cons  3547.53    0.00
304     li_amogs_size_mn_mn  3285.40    0.00
456             li_wsum_iqr  2995.18    0.00
780           v_cv_deg_vars  2978.31   -0.51
900            v_ratio_vars  2915.94   -0.78
432             li_sepr_max  2909.36   -0.93
680              pb_skew_mn  2898.03    0.00
692             pb_wsum_sum  2716.45   -0.01
102       c_max_domdeg_cons  2439.12    1.04
440             li_skew_ent  2371.79    0.39
292  li_amogs_maxw_skew_ent  2273.31   -0.28
656               pb_sep_mn  2252.35    0.80
560      pb_k_amogs_prod_mn  2247.26    0.09
328     li_k_amogs_prod_iqr  2241.15    0.05
668             pb_skew_ent  1771.45    0.00 

class combi by median
           